In [15]:
# from jetstream_hugo.definitions import *
# from jetstream_hugo.plots import *
# from jetstream_hugo.data import *
# # from jetstream_hugo.anyspell import *
# from jetstream_hugo.jet_finding import *
# from jetstream_hugo.clustering import *

# %load_ext autoreload
# %autoreload 2
# %matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from collections.abc import Sequence
from typing import Tuple
from itertools import pairwise, product
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed
from jetstream_hugo.definitions import DATADIR, KAPPA, compute
from jetstream_hugo.data import standardize, flatten_by, extract
import numpy as np 
from netCDF4 import Dataset
import xarray as xr
import h5py
import argparse
import traceback
import intake
import intake_esm


varname_to_search_dict = {
    "high_wind": ["U", "V", "T"],
    "mid_wind": ["U500", "V500"],
    "PRECL": "PRECL",
    "TS": "TS",
}
levels_dict = {
    "high_wind": list(range(14, 18)),
    "mid_wind": "all",
    "PRECL": "all",
    "TS": "all",
}
experiment_dict = {
    "past": "historical",
    "future": "ssp370",
}
yearbounds_dict = {
    "past": np.arange(1960, 2021, 10),
    "future": np.arange(2045, 2106, 10),
}
for key, val in yearbounds_dict.items():
    yearbounds_dict[key][-1] = val[-1] - 5

# parser=argparse.ArgumentParser(description="sample argument parser")
# parser.add_argument("period", choices=["past", "future"])
# parser.add_argument("variable", choices=["high_wind", "mid_wind", "PRECL"])
# args=parser.parse_args()
# period = args.period
# variable = args.variable
period = "past"
variable = "high_wind"

url = 'https://raw.githubusercontent.com/NCAR/cesm2-le-aws/main/intake-catalogs/aws-cesm2-le.json'
component = "atm"
experiment = experiment_dict[period]
yearbounds = yearbounds_dict[period]
varname_to_search = varname_to_search_dict[variable]
levels = levels_dict.get(variable, None)
frequency = "daily"
forcing_variant = "cmip6"
season = None
minlon = -180
maxlon = 180
minlat = 0
maxlat = 90
members = "all"
reduce_da = True
Path(f"{DATADIR}/CESM2/{variable}/{period}").mkdir(exist_ok=True)

indexers = [component, experiment, frequency, forcing_variant]
indexers = [np.atleast_1d(indexer) for indexer in indexers]
indexers = [[str(idx_) for idx_ in indexer] for indexer in indexers]
indexers = list(product(*indexers))
ensemble_keys = [".".join(indexer) for indexer in indexers]

catalog = intake.open_esm_datastore(url)
catalog_subset = catalog.search(
    variable=varname_to_search,
    component=component,
    experiment=experiment,
    frequency=frequency,
    forcing_variant=forcing_variant,
)
dsets = catalog_subset.to_dataset_dict(
    xarray_open_kwargs={"consolidated": False},
    storage_options={"anon": True}
)
ds = dsets[ensemble_keys[0]]
ds = standardize(ds)
members = np.unique(ds.member.values)
def downloader(ds_: xr.Dataset, varname: str | list, period: str, member: str, bounds: Tuple[int]):
    opath = Path(f"{DATADIR}/CESM2/{variable}/{period}/{member}.{bounds[0]}-{bounds[1]}.nc")
    if opath.is_file():
        return f"{member} {bounds[0]}-{bounds[1]} already exists"
    ds = extract(
        ds_,
        period=bounds,
        season=season,
        minlon=minlon,
        maxlon=maxlon,
        minlat=minlat,
        maxlat=maxlat,
        levels=levels_dict[varname],
        members=member,
    )
    if varname in ["high_wind", "mid_wind"]:
        ds["s"] = ds["s"] = np.sqrt(ds["u"] ** 2 + ds["v"] ** 2)
    ds = compute(ds, progress_flag=True, n_workers=1)
    if varname == "high_wind":
        ds = flatten_by(ds, "s")
        ds["theta"] = (1000 / ds["lev"]) ** KAPPA * ds["t"]
        ds = ds.drop_vars("t")
    for varname in (list(ds.data_vars) + ["lat", "lon"]):
        ds[varname] = ds[varname].astype(np.float32)
    ds = ds.to_netcdf(opath)
    return f"Retrieved {member} {bounds[0]}-{bounds[1]}"
    
bounds = [(y0, y1) for y0, y1 in pairwise(yearbounds_dict[period])]
with ThreadPoolExecutor(max_workers=10) as executor:
    futures = [
        executor.submit(downloader, ds, variable, period, member, bounds_) for member in members for bounds_ in bounds
    ]
    for f in as_completed(futures):
        try:
            print(f.result())
        except Exception as e:
            print(traceback.format_exc())
            print("could not retrieve")                



--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.frequency.forcing_variant'


/storage/homefs/hb22g102/miniforge3/envs/env11_2/lib/python3.11/site-packages/dask/array/core.py:5028: PerformanceWarning: Increasing number of chunks by factor of 50
  result = blockwise(


[                                        ] | 0% Completed | 205.56 ss

[                                        ] | 0% Completed | 227.81 s

KeyboardInterrupt: 

[                                        ] | 0% Completed | 208.17 s

In [4]:
[(y0, y1) for y0, y1 in pairwise(yearbounds_dict[period])]

[(np.int64(1960), np.int64(1970)),
 (np.int64(1970), np.int64(1980)),
 (np.int64(1980), np.int64(1990)),
 (np.int64(1990), np.int64(2000)),
 (np.int64(2000), np.int64(2010)),
 (np.int64(2010), np.int64(2020))]

In [2]:
downloader(ds, variable, period, members[0])

[                                        ] | 0% Completed | 70.80 sms


KeyboardInterrupt: 

In [12]:
ds

<xarray.Dataset> Size: 128TB
Dimensions:    (lat: 192, lev: 32, member: 50, time: 60225, lon: 288, nbnd: 2)
Coordinates:
  * lat        (lat) float64 2kB -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lev        (lev) float64 256B 3.643 7.595 14.36 24.61 ... 957.5 976.3 992.6
  * member     (member) <U12 2kB 'r10i1181p1f1' 'r10i1231p1f1' ... 'r9i1301p1f1'
  * time       (time) object 482kB 1850-01-01 12:00:00 ... 2014-12-31 12:00:00
    time_bnds  (time, nbnd) object 964kB dask.array<chunksize=(10, 1), meta=np.ndarray>
  * lon        (lon) float64 2kB -180.0 -178.8 -177.5 ... 176.2 177.5 178.8
Dimensions without coordinates: nbnd
Data variables:
    t          (member, time, lev, lat, lon) float64 43TB dask.array<chunksize=(1, 10, 32, 192, 288), meta=np.ndarray>
    u          (member, time, lev, lat, lon) float64 43TB dask.array<chunksize=(1, 10, 32, 192, 288), meta=np.ndarray>
    v          (member, time, lev, lat, lon) float64 43TB dask.array<chunksize=(1, 10, 32, 192, 288), meta=np.ndarray>
Attributes: (12/16)
    Conventions:                       CF-1.0
    logname:                           sunseon
    model_doi_url:                     https://doi.org/10.5065/D67H1H0V
    source:                            CAM
    time_period_freq:                  day_1
    topography_file:                   /mnt/lustre/share/CESM/cesm_input/atm/...
    ...                                ...
    intake_esm_attrs:vertical_levels:  32.0
    intake_esm_attrs:spatial_domain:   global
    intake_esm_attrs:start_time:       1850-01-01 12:00:00
    intake_esm_attrs:end_time:         2014-12-31 12:00:00
    intake_esm_attrs:_data_format_:    zarr
    intake_esm_dataset_key:            atm.historical.daily.cmip6

# arco-era5 tests

In [ ]:
ds = xr.open_zarr(
    "gs://gcp-public-data-arco-era5/ar/full_37-1h-0p25deg-chunk-1.zarr-v3",
    chunks=None,
    storage_options=dict(token="anon"),
)
ar_full_37_1h = ds.sel(
    time=slice(ds.attrs["valid_time_start"], ds.attrs["valid_time_stop"])
)

base_ds = (
    ar_full_37_1h[["u_component_of_wind", "v_component_of_wind"]]
    .sel(
        time=ar_full_37_1h.time.dt.hour % 6 == 0,
        latitude=ar_full_37_1h.latitude >= 0,
        level=[175, 200, 225, 250, 300, 350],
    )
    .isel(longitude=slice(None, None, 2), latitude=slice(None, None, 2))
)

base_path = Path(f"{DATADIR}/ERA5/plev/flat_wind/dailymean")
for year in YEARS:
    for month in range(1, 13):
        month_str = str(month).zfill(2)
        opath = base_path.joinpath(f"{year}{month_str}.nc")
        if opath.is_file():
            continue
        ds = compute(
            base_ds.sel(
                time=(base_ds.time.dt.year == year) & (base_ds.time.dt.month == month)
            ),
            progress=True,
        )
        ds = standardize(ds)
        ds["s"] = np.sqrt(ds["u"] ** 2 + ds["v"] ** 2)
        ds = flatten_by(ds, "s")
        ds.to_netcdf(opath)
        print(f"Completed {year}{month}")

In [ ]:
ds = xr.open_zarr(
    "gs://gcp-public-data-arco-era5/ar/full_37-1h-0p25deg-chunk-1.zarr-v3",
    chunks=None,
    storage_options=dict(token="anon"),
)
ar_full_37_1h = ds.sel(
    time=slice(ds.attrs["valid_time_start"], ds.attrs["valid_time_stop"])
)

temp_full = (
    ar_full_37_1h["temperature"]
    .sel(
        time=ar_full_37_1h.time.dt.hour % 6 == 0,
        latitude=ar_full_37_1h.latitude >= 0,
        level=[175, 200, 225, 250, 300, 350],
    )
    .isel(longitude=slice(None, None, 2), latitude=slice(None, None, 2))
)

temp_full = standardize(temp_full)

In [ ]:
ds = xr.open_zarr(
    "gs://gcp-public-data-arco-era5/ar/full_37-1h-0p25deg-chunk-1.zarr-v3",
    chunks=None,
    storage_options=dict(token="anon"),
)
ar_full_37_1h = ds.sel(
    time=slice(ds.attrs["valid_time_start"], ds.attrs["valid_time_stop"])
)

temp_full = (
    ar_full_37_1h["temperature"]
    .sel(
        time=ar_full_37_1h.time.dt.hour % 6 == 0,
        latitude=ar_full_37_1h.latitude >= 0,
        level=[175, 200, 225, 250, 300, 350],
    )
    .isel(longitude=slice(None, None, 2), latitude=slice(None, None, 2))
)

temp_full = standardize(temp_full)

orig_path = Path(f"{DATADIR}/ERA5/plev/flat_wind/dailymean")
base_path = Path(f"{DATADIR}/ERA5/plev/flat_wind/dailymean_2")
for year in tqdm(YEARS):
    for month in trange(1, 13, leave=False):
        month_str = str(month).zfill(2)
        opath = base_path.joinpath(f"{year}{month_str}.nc")
        if opath.is_file():
            continue
        ipath = orig_path.joinpath(f"{year}{month_str}.nc")
        ds = xr.open_dataset(ipath)
        this_temp = temp_full.sel(time=ds.time.values, lev=ds["lev"])
        this_temp = this_temp * (1000 / this_temp.lev) ** KAPPA
        this_temp = this_temp.reset_coords("lev", drop=True)
        ds["theta"] = compute(this_temp, progress_flag=True)
        ds.to_netcdf(opath)

# new pvs das

In [ ]:
import polars_st as st

In [ ]:
def to_xarray(events: st.GeoDataFrame, dummy_da: xr.DataArray, varname: str):
    dummy_da = dummy_da.rename("dummy")
    da_df = pl.from_pandas(dummy_da.to_dataframe().reset_index())
    orig_times = da_df["time"]
    timedtype = orig_times.dtype
    da_df = da_df.drop("time", "dummy")
    da_df = da_df.unique(["lat", "lon"]).sort(["lat", "lon"]).with_columns(geometry=st.from_xy("lon", "lat"))
    da_df = st.GeoDataFrame(da_df)
    events = events.with_columns(pl.col("geometry").st.buffer(0.25))
    events = (
        events.select(["date", "geometry", varname])
        .cast({varname: pl.Float32})
        .rename({"date": "time"})
    )
    events = events.cast({"time": timedtype})
    events = events.filter(pl.col("time").is_in(orig_times))
    dummy_da = xr.zeros_like(dummy_da, dtype=np.float32)
    events = events.st.sjoin(da_df, on="geometry", how="inner", predicate="contains")
    events = events.unique(["time", "lon", "lat"])
    events_da = xr.DataArray.from_series(
        events[["time", varname, "lat", "lon"]]
        .to_pandas()
        .set_index(["time", "lat", "lon"])[varname]
    ).fillna(0)
    dummy_da.loc[
        {
            "time": events_da.time.values,
            "lat": events_da.lat.values,
            "lon": events_da.lon.values,
        }
    ] = events_da
    return dummy_da

In [ ]:
all_events = {}
for level in trange(310, 365, 5):
    events = st.from_geopandas(gpd.read_parquet(f"/storage/workspaces/giub_meteo_impacts/ci01/ERA5/RWB_index/era5_pv_streamers_{level}K_1959-2022.parquet"))

    tropospheric = events.filter(pl.col("mean_var") < pl.col("level"))
    anticyclonic = tropospheric.filter(pl.col("intensity") >= pl.col("level"))
    cyclonic = tropospheric.filter(pl.col("intensity") < pl.col("level"))
    
    all_events[level] = {"anti": anticyclonic, "cycl": cyclonic}

In [ ]:
for year in YEARS:
    # for month in range(1, 13):
    ofile_anti = Path(f"{DATADIR}/ERA5/thetalev/apvs/6H/{year}.nc")
    ofile_cycl = Path(f"{DATADIR}/ERA5/thetalev/cpvs/6H/{year}.nc")
    if ofile_cycl.is_file() and ofile_anti.is_file():
        continue
    time_mask = (TIMERANGE.year == year)# & (TIMERANGE.month == month)
    coords = {
        "time": TIMERANGE[time_mask],
        "lat": np.arange(0, 90.5, .5),
        "lon": np.arange(-180, 180, .5),
    }
    shape = [len(co) for co in coords.values()]
    dummy_da = xr.DataArray(np.zeros(shape), coords=coords)
    anti_all_levs = {}
    cycl_all_levs = {}
    for lev, events in tqdm(all_events.items(), total=11):
        anti_all_levs[lev] = to_xarray(events["anti"], dummy_da, "intensity")
        cycl_all_levs[lev] = to_xarray(events["cycl"], dummy_da, "intensity")
    anti_all_levs = xr.concat(anti_all_levs.values(), dim="lev").assign_coords(lev=list(anti_all_levs))
    cycl_all_levs = xr.concat(cycl_all_levs.values(), dim="lev").assign_coords(lev=list(cycl_all_levs))
    anti_all_levs.to_netcdf(ofile_anti)
    cycl_all_levs.to_netcdf(ofile_cycl)

# CESM

In [7]:
from itertools import product
from pathlib import Path
import numpy as np
import xarray as xr
import intake
from jetstream_hugo.data import extract, flatten_by, standardize
from jetstream_hugo.definitions import DATADIR, compute, KAPPA
import argparse

varname_to_search_dict = {
    "high_wind": ["U", "V", "T"],
    "mid_wind": ["U", "V"],
}
levels_dict = {
    "high_wind": list(range(14, 18)),
    "mid_wind": [20],
}
experiment_dict = {
    "past": "historical",
    "future": "ssp370",
}
yearbounds_dict = {
    "past": (1964, 2015),
    "future": (2049, 2100),
}

# parser=argparse.ArgumentParser(description="sample argument parser")
# parser.add_argument("period", choices=["past", "future"])
# parser.add_argument("variable", choices=["high_wind", "mid_wind", "PRECL"])
# args=parser.parse_args()
# period = args.period
# variable = args.variable
period = "past"
variable = "high_wind"

url = 'https://raw.githubusercontent.com/NCAR/cesm2-le-aws/main/intake-catalogs/aws-cesm2-le.json'
component = "atm"
experiment = experiment_dict[period]
yearbounds = yearbounds_dict[period]
varname_to_search = varname_to_search_dict[variable]
levels = levels_dict.get(variable, None)
out_path = Path(f"{DATADIR}/CESM2/{variable}/{period}.zarr")
frequency = "daily"
forcing_variant = "cmip6"
season = None
minlon = -180
maxlon = 180
minlat = 0
maxlat = 90
members = "all"
reduce_da = True

indexers = [component, experiment, frequency, forcing_variant]
indexers = [np.atleast_1d(indexer) for indexer in indexers]
indexers = [[str(idx_) for idx_ in indexer] for indexer in indexers]
indexers = list(product(*indexers))
ensemble_keys = [".".join(indexer) for indexer in indexers]

catalog = intake.open_esm_datastore(url)
catalog_subset = catalog.search(
    variable=varname_to_search,
    component=component,
    experiment=experiment,
    frequency=frequency,
    forcing_variant=forcing_variant,
)
dsets = catalog_subset.to_dataset_dict(
    xarray_open_kwargs={"consolidated": False},
    storage_options={"anon": True}
)
ds = dsets[ensemble_keys[0]]

ds = standardize(ds)

if out_path.is_dir():
    ds_on_disk = xr.open_dataset(out_path)
    years_on_disk = np.unique(ds_on_disk.time.dt.year.values)
else:
    years_on_disk = []

for i, year in enumerate(range(*yearbounds)):
    if year in years_on_disk:
        continue
    ds_ = extract(
        ds,
        period=[year],
        season=season,
        minlon=minlon,
        maxlon=maxlon,
        minlat=minlat,
        maxlat=maxlat,
        levels=levels,
        # members=[i],
    )
    ds_["s"] = np.sqrt(ds_["u"] ** 2 + ds_["v"] ** 2)
    # ds_ = compute(ds_, progress_flag=True)
    # ds_ = flatten_by(ds_, "s")
    if variable == "high_wind":
        ds_["theta"] = (1000 / ds_["lev"]) ** KAPPA * ds_["t"]
        ds_ = ds_.drop_vars("t")
    break
    # for varname in (list(ds_.data_vars) + ["lat", "lon"]):
    #     ds_[varname] = ds_[varname].astype(np.float32)
    # if i == 0:
    #     ds_.to_zarr(out_path, mode="w")
    # else:
    #     ds_.to_zarr(out_path, append_dim="time")
    # print(year)


--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.frequency.forcing_variant'


/storage/homefs/hb22g102/miniforge3/envs/env11_2/lib/python3.11/site-packages/dask/array/core.py:5028: PerformanceWarning: Increasing number of chunks by factor of 50
  result = blockwise(
/storage/homefs/hb22g102/jetstream_hugo/src/jetstream_hugo/data.py:138: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  da["time"] = da.indexes["time"].to_datetimeindex()


In [ ]:
ds_

<xarray.Dataset> Size: 65GB
Dimensions:    (lat: 96, lev: 4, member: 50, time: 365, lon: 288, nbnd: 2)
Coordinates:
  * lat        (lat) float64 768B 0.4712 1.414 2.356 3.298 ... 88.12 89.06 90.0
  * lev        (lev) float64 32B 197.9 232.8 273.9 322.2
  * member     (member) <U12 2kB 'r10i1181p1f1' 'r10i1231p1f1' ... 'r9i1301p1f1'
  * time       (time) datetime64[ns] 3kB 1964-01-01T12:00:00 ... 1964-12-31T1...
    time_bnds  (time, nbnd) object 6kB dask.array<chunksize=(9, 1), meta=np.ndarray>
  * lon        (lon) float64 2kB -180.0 -178.8 -177.5 ... 176.2 177.5 178.8
Dimensions without coordinates: nbnd
Data variables:
    t          (member, time, lev, lat, lon) float64 16GB dask.array<chunksize=(1, 9, 4, 96, 288), meta=np.ndarray>
    u          (member, time, lev, lat, lon) float64 16GB dask.array<chunksize=(1, 9, 4, 96, 288), meta=np.ndarray>
    v          (member, time, lev, lat, lon) float64 16GB dask.array<chunksize=(1, 9, 4, 96, 288), meta=np.ndarray>
    s          (member, time, lev, lat, lon) float64 16GB dask.array<chunksize=(1, 9, 4, 96, 288), meta=np.ndarray>
Attributes: (12/17)
    Conventions:                       CF-1.0
    logname:                           sunseon
    model_doi_url:                     https://doi.org/10.5065/D67H1H0V
    source:                            CAM
    time_period_freq:                  day_1
    topography_file:                   /mnt/lustre/share/CESM/cesm_input/atm/...
    ...                                ...
    intake_esm_attrs:spatial_domain:   global
    intake_esm_attrs:start_time:       1850-01-01 12:00:00
    intake_esm_attrs:end_time:         2014-12-31 12:00:00
    intake_esm_attrs:_data_format_:    zarr
    intake_esm_dataset_key:            atm.historical.daily.cmip6
    orig_lev:                          [14, 15, 16, 17]

In [7]:
from jetstream_hugo.definitions import *
from jetstream_hugo.data import *
import intake
url = 'https://raw.githubusercontent.com/NCAR/cesm2-le-aws/main/intake-catalogs/aws-cesm2-le.json'
varname_to_search = ["U", "V", "T"]
varname = "high_wind"
component = "atm"
experiment = "historical"
# experiment = "ssp370"
frequency = "daily"
forcing_variant = "cmip6"
# period = (1980, 2009)
# period = (2070, 2099)
season = None
minlon = -180
maxlon = 180
minlat = 0
maxlat = 90
levels = list(range(14, 18))
members = "all"
reduce_da = True

indexers = [component, experiment, frequency, forcing_variant]
indexers = [np.atleast_1d(indexer) for indexer in indexers]
indexers = [[str(idx_) for idx_ in indexer] for indexer in indexers]
indexers = list(product(*indexers))
ensemble_keys = [".".join(indexer) for indexer in indexers]

basepath = Path(f"{DATADIR}/CESM2/{varname}")
catalog = intake.open_esm_datastore(url)
catalog_subset = catalog.search(
    variable=varname_to_search,
    component=component,
    experiment=experiment,
    frequency=frequency,
    forcing_variant=forcing_variant,
)
dsets = catalog_subset.to_dataset_dict(
    xarray_open_kwargs={"consolidated": False},
    storage_options={"anon": True}
)
ds = dsets[ensemble_keys[0]]

ds = standardize(ds)


--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.frequency.forcing_variant'


/storage/homefs/hb22g102/miniforge3/envs/env11_2/lib/python3.11/site-packages/dask/array/core.py:5028: PerformanceWarning: Increasing number of chunks by factor of 50
  result = blockwise(
/storage/homefs/hb22g102/jetstream_hugo/src/jetstream_hugo/data.py:138: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  da["time"] = da.indexes["time"].to_datetimeindex()


In [8]:
for year in range(1965, 2016):
    ds_ = extract(
        ds,
        period=[year],
        season=season,
        minlon=minlon,
        maxlon=maxlon,
        minlat=minlat,
        maxlat=maxlat,
        levels=levels,
        # members=[i],
    )
    ds_["s"] = np.sqrt(ds_["u"] ** 2 + ds_["v"] ** 2)
    ds_ = compute(ds_, progress_flag=True)
    ds_ = flatten_by(ds_, "s")
    ds_["theta"] = (1000 / ds_["lev"]) ** KAPPA * ds_["t"]
    ds_ = ds_.drop_vars("t")
    for varname in (list(ds_.data_vars) + ["lat", "lon"]):
        ds_[varname] = ds_[varname].astype(np.float32)
    #     if i == 0:
    #         ds_.to_zarr(f"{DATADIR}/CESM2/test.zarr", mode="w")
    #     elif i == 1:
    #         ds_.to_zarr(f"{DATADIR}/CESM2/test.zarr", append_dim="member")
    #     else:
    #         break
        break
    break

[                                        ] | 1% Completed | 74.61 sms


KeyboardInterrupt: 

In [10]:
ds.time.dt.year

<xarray.DataArray 'year' (time: 60225)> Size: 482kB
array([1850, 1850, 1850, ..., 2014, 2014, 2014])
Coordinates:
  * time     (time) datetime64[ns] 482kB 1850-01-01T12:00:00 ... 2014-12-31T1...

In [53]:
ds_

<xarray.Dataset> Size: 97GB
Dimensions:    (lat: 96, lev: 6, member: 50, time: 365, lon: 288, nbnd: 2)
Coordinates:
  * lat        (lat) float64 768B 0.4712 1.414 2.356 3.298 ... 88.12 89.06 90.0
  * lev        (lev) float64 48B 168.2 197.9 232.8 273.9 322.2 379.1
  * member     (member) <U12 2kB 'r10i1181p1f1' 'r10i1231p1f1' ... 'r9i1301p1f1'
  * time       (time) datetime64[ns] 3kB 1945-01-01T12:00:00 ... 1945-12-31T1...
    time_bnds  (time, nbnd) object 6kB dask.array<chunksize=(9, 1), meta=np.ndarray>
  * lon        (lon) float64 2kB -180.0 -178.8 -177.5 ... 176.2 177.5 178.8
Dimensions without coordinates: nbnd
Data variables:
    t          (member, time, lev, lat, lon) float64 24GB dask.array<chunksize=(1, 9, 6, 96, 288), meta=np.ndarray>
    u          (member, time, lev, lat, lon) float64 24GB dask.array<chunksize=(1, 9, 6, 96, 288), meta=np.ndarray>
    v          (member, time, lev, lat, lon) float64 24GB dask.array<chunksize=(1, 9, 6, 96, 288), meta=np.ndarray>
    s          (member, time, lev, lat, lon) float64 24GB dask.array<chunksize=(1, 9, 6, 96, 288), meta=np.ndarray>
Attributes: (12/17)
    Conventions:                       CF-1.0
    logname:                           sunseon
    model_doi_url:                     https://doi.org/10.5065/D67H1H0V
    source:                            CAM
    time_period_freq:                  day_1
    topography_file:                   /mnt/lustre/share/CESM/cesm_input/atm/...
    ...                                ...
    intake_esm_attrs:spatial_domain:   global
    intake_esm_attrs:start_time:       1850-01-01 12:00:00
    intake_esm_attrs:end_time:         2014-12-31 12:00:00
    intake_esm_attrs:_data_format_:    zarr
    intake_esm_dataset_key:            atm.historical.daily.cmip6
    orig_lev:                          [13, 14, 15, 16, 17, 18]

In [47]:
ds_.to_zarr(f"{DATADIR}/CESM2/test.zarr", append_dim="member")

In [42]:
ds_ = ds_.expand_dims("member").copy()

In [48]:
xr.open_dataset(f"{DATADIR}/CESM2/test.zarr")

/storage/homefs/hb22g102/miniforge3/envs/env11_2/lib/python3.11/site-packages/xarray/backends/api.py:651: RuntimeWarning: 'netcdf4' fails while guessing
  engine = plugins.guess_engine(filename_or_obj)
/storage/homefs/hb22g102/miniforge3/envs/env11_2/lib/python3.11/site-packages/xarray/backends/api.py:651: RuntimeWarning: 'h5netcdf' fails while guessing
  engine = plugins.guess_engine(filename_or_obj)
/storage/homefs/hb22g102/miniforge3/envs/env11_2/lib/python3.11/site-packages/xarray/backends/api.py:651: RuntimeWarning: 'scipy' fails while guessing
  engine = plugins.guess_engine(filename_or_obj)


<xarray.Dataset> Size: 727MB
Dimensions:    (lat: 96, member: 2, time: 365, lon: 288, nbnd: 2)
Coordinates:
  * lat        (lat) float64 768B 0.4712 1.414 2.356 3.298 ... 88.12 89.06 90.0
  * lon        (lon) float64 2kB -180.0 -178.8 -177.5 ... 176.2 177.5 178.8
  * member     (member) <U12 96B 'r10i1181p1f1' 'r10i1181p1f2'
  * time       (time) datetime64[ns] 3kB 1945-01-01T12:00:00 ... 1945-12-31T1...
    time_bnds  (time, nbnd) object 6kB ...
Dimensions without coordinates: nbnd
Data variables:
    lev        (member, time, lat, lon) float32 81MB ...
    s          (member, time, lat, lon) float64 161MB ...
    theta      (member, time, lat, lon) float64 161MB ...
    u          (member, time, lat, lon) float64 161MB ...
    v          (member, time, lat, lon) float64 161MB ...
Attributes: (12/18)
    Conventions:                       CF-1.0
    flattened:                         1
    intake_esm_attrs:_data_format_:    zarr
    intake_esm_attrs:component:        atm
    intake_esm_attrs:end_time:         2014-12-31 12:00:00
    intake_esm_attrs:experiment:       historical
    ...                                ...
    logname:                           sunseon
    model_doi_url:                     https://doi.org/10.5065/D67H1H0V
    orig_lev:                          [168.22507977485657, 197.9080867022276...
    source:                            CAM
    time_period_freq:                  day_1
    topography_file:                   /mnt/lustre/share/CESM/cesm_input/atm/...

In [4]:
da = compute(da, progress_flag=True)

[########################################] | 100% Completed | 11m 31s


In [5]:
da.to_zarr("/storage/workspaces/giub_meteo_impacts/ci01/CESM2/TS/past.zarr", mode="w")

In [ ]:
from pathlib import Path
import numpy as np
import xarray as xr
basepath = Path("/storage/workspaces/giub_meteo_impacts/ci01/CESM2/flat_wind")
paths = list(basepath.iterdir())
paths = [path for path in paths if path.suffix == ".nc" and path.name != "ds.nc"]
parts = [path.name.split(".")[0].split("-") for path in paths]
parts = np.asarray(parts)
sorted_order = np.argsort([memb.replace("r10", "r0") for memb in parts[:, 0]])
parts = parts[sorted_order]
paths = [paths[i] for i in sorted_order]
all_members = np.unique(parts[:, 0])
all_years = np.unique(parts[:, 1])

not_here = []
here = []
for year in all_years:
    for member in all_members:
        potential_path = basepath.joinpath(f"{member}-{year}.nc")
        if potential_path.is_file():
            here.append(potential_path)
        else:
            not_here.append(potential_path)
len(here)

from itertools import groupby
paths_to_load = []
valid_ensembles = []
for key, indices in groupby(range(len(parts)), lambda i: parts[i][0]):
    indices = list(indices)
    group = parts[indices]
    these_paths = [paths[i] for i in indices]
    years = np.asarray(list([g[1] for g in group]))
    if len(years) == 60:
        paths_to_load.append(these_paths)
        valid_ensembles.append(key)
    else:
        print(key, len(years))

In [ ]:
from tqdm import tqdm
ds = []
for ptl in tqdm(paths_to_load):
    ds_ = []
    for p in ptl:
        this = xr.open_dataset(p)
        this = this.reset_coords("time_bnds", drop=True).drop_dims("nbnd")
        ds_.append(this)
    ds.append(xr.concat(ds_, dim="time"))
ds = xr.concat(ds, dim="member")
# ds = xr.concat([xr.concat([xr.open_dataset[ptl_] for ptl_ in ptl], dim="time") for ptl in paths_to_load], dim="member")

In [ ]:
import dask
from dask.distributed import progress, Client
from jetstream_hugo.definitions import COMPUTE_KWARGS
client = Client(**COMPUTE_KWARGS)
dask.persist(ds)
progress(ds, notebook=False)
ds = dask.compute(ds)

In [ ]:
ds = ds[0]
to_comp = ds.to_zarr(f"/storage/workspaces/giub_meteo_impacts/ci01/CESM2/flat_wind/ds.zarr", compute=False, encoding={var: {"chunks": (-1, 100, -1, -1)} for var in ds.data_vars}, mode="w")
dask.persist(to_comp)
progress(to_comp, notebook=False)
dask.compute(to_comp)